In [1]:
!pip install -q bitsandbytes accelerate peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import pandas as pd
from sklearn.model_selection import train_test_split

2025-11-22 14:56:25.282252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763823385.671595      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763823385.791627      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [3]:
model_id = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [4]:
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 33,030,144 || all params: 4,055,498,240 || trainable%: 0.8145


In [5]:
# === 指令模板 ===
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a local resident of a region around the world. Answer the following common-sense question based on real-world knowledge in that region. Respond in clear, standard English. Be concise and factual. Provide only the essential answer without any explanation, introduction, or punctuation.

### Input:
{}

### Response:
{}"""

prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a local resident of a region around the world. Answer the following common-sense question based on real-world knowledge in that region. Respond in clear, standard English. Be concise and factual. Provide only the essential answer without any explanation, introduction, or punctuation.

### Input:
{}

### Response:"""


In [6]:
EOS_TOKEN = tokenizer.eos_token
# === 数据格式化函数 ===
def formatting_prompts_func(examples):
    questions = examples["question"]
    answers = examples["answer"]
    outputs = []
    for q, a in zip(questions, answers):
        text = alpaca_prompt.format(q, a) + EOS_TOKEN
        outputs.append(text)
    return {"text": outputs}

# === 加载训练数据 ===
build_data = pd.read_csv("/kaggle/input/train-data/build_data.csv",header=0, delimiter="\t", quoting=3)
test_df = pd.read_csv("/kaggle/input/train-data/output_with_translation.tsv", header=0, delimiter="\t", quoting=3)
train_df, val_df = train_test_split(build_data, test_size=0.2, random_state=42)

train_dict = {'answer': train_df["answer_en"], 'question': train_df['question_en']}
val_dict = {'answer': val_df["answer_en"], 'question': val_df['question_en']}
test_dict = {"question": test_df['question_en']}

train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)
test_dataset = Dataset.from_dict(test_dict)

# 格式化
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
val_dataset = val_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/1578 [00:00<?, ? examples/s]

Map:   0%|          | 0/395 [00:00<?, ? examples/s]

In [7]:
print(train_dataset[0]["text"])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a local resident of a region around the world. Answer the following common-sense question based on real-world knowledge in that region. Respond in clear, standard English. Be concise and factual. Provide only the essential answer without any explanation, introduction, or punctuation.

### Input:
Are meals typically served with rice in West Java, Indonesia?

### Response:
Yes<|im_end|>


In [8]:
os.makedirs('/kaggle/working/checkpoint', exist_ok=True)
training_args = TrainingArguments(
    output_dir="/kaggle/working/checkpoint",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    optim="paged_adamw_8bit",
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    seed=3407,
    report_to="none",
)
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


Adding EOS to train dataset:   0%|          | 0/1578 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1578 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1578 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/395 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/395 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/395 [00:00<?, ? examples/s]

In [9]:

trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1,0.090600,0.086067,0.088215,166658.000000,0.973335
2,0.080800,0.083296,0.083251,333316.000000,0.973816
3,0.070300,0.079952,0.079150,499974.000000,0.974636


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=594, training_loss=0.11887684318955097, metrics={'train_runtime': 8016.2723, 'train_samples_per_second': 0.591, 'train_steps_per_second': 0.074, 'total_flos': 1.162431701342208e+16, 'train_loss': 0.11887684318955097, 'epoch': 3.0})

In [10]:
model.save_pretrained("/kaggle/working/final_lora")
tokenizer.save_pretrained("/kaggle/working/final_lora")

('/kaggle/working/final_lora/tokenizer_config.json',
 '/kaggle/working/final_lora/special_tokens_map.json',
 '/kaggle/working/final_lora/chat_template.jinja',
 '/kaggle/working/final_lora/vocab.json',
 '/kaggle/working/final_lora/merges.txt',
 '/kaggle/working/final_lora/added_tokens.json',
 '/kaggle/working/final_lora/tokenizer.json')

In [11]:
from tqdm import tqdm

model.half()
# === 推理 ===
test_df = pd.read_csv("/kaggle/input/train-data/output_with_translation.tsv", sep='\t')

test_questions = test_df["question_en"].tolist()
test_ids = test_df["id"].tolist()

predictions = []

for q in tqdm(test_questions, desc="Inference"):
    prompt = alpaca_prompt.format(q, "") 
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024,
        padding=False,
        add_special_tokens=True,
    ).to(model.device)
    
    input_length = inputs["input_ids"].shape[1]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True,
        )

    generated_tokens = outputs[0][input_length:]
    answer = tokenizer.decode(
        generated_tokens,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    ).strip()
    
    predictions.append(answer)


Inference: 100%|██████████| 148/148 [02:17<00:00,  1.07it/s]


In [12]:
# === 保存结果 ===
result_df = pd.DataFrame({
    "id": test_ids,
    "answer_en": predictions
})

result_df.to_csv("/kaggle/working/saq_instruction_tuning.csv", index=False, sep='\t')